In [98]:
import numpy as np
from scipy import stats
from scipy.stats import norm

In [99]:
# ex 1. Crude monte carlo simulator

def confidence_interval(confidence, means, point_estimate):
    std_error = stats.sem(means)
    return stats.t.interval(confidence, df=len(means)-1, loc=point_estimate, scale=std_error)


def crude_monte_carlo(integral_range: tuple, samples: int, func):
    a, b = integral_range
    return func(a, b, samples)

In [100]:
# mean of integral of e^x from 0 to 1
integral_range = (0,1)
samples = 100
f = lambda a, b, samples: np.exp(np.random.uniform(a, b, samples))
X = crude_monte_carlo(integral_range, samples, f)

X_point_estimate = np.mean(X)
ci = confidence_interval(0.95, X, X_point_estimate)
print("Exercise 1:")
print("Point estimate:", X_point_estimate)
print(f"Confidence interval: ({ci[0]:.5f}, {ci[1]:.5f})")

Exercise 1:
Point estimate: 1.7317128301304363
Confidence interval: (1.63298, 1.83044)


In [101]:
# Exercise 2
def antithetic_variables(integral_range, samples):
    Ui = np.random.uniform(low=integral_range[0], high=integral_range[1], size=samples)
    Y = (np.exp(Ui) + np.exp(1-Ui)) / 2
    return np.array(Y)

In [102]:
Y = antithetic_variables(integral_range, samples//2)

Y_point_estimate = np.mean(Y)
ci = confidence_interval(0.95, Y, Y_point_estimate)
print("Exercise 2:")
print("Point estimate:", Y_point_estimate)
print(f"Confidence interval: ({ci[0]:.5f}, {ci[1]:.5f})")

Exercise 2:
Point estimate: 1.7134401999183921
Confidence interval: (1.69617, 1.73071)


In [103]:
# Exercise 3
def control_variate(X, Y, mean_y):
    c = -np.cov(X, Y, ddof=1)[0][1] / np.var(Y, ddof=1)
    Z = X + c * (Y - mean_y)
    var_Z = np.var(X, ddof=1) - np.cov(X,Y, ddof=1)[0][1] ** 2 / np.var(Y, ddof=1)
    return Z, var_Z

In [104]:
# Example from slides
U = np.random.uniform(0, 1, samples)
Z, var_Z = control_variate(np.exp(U), U, np.mean(U))

Z_point_estimate = np.mean(Z)
print("Exercise 3 with variables from slides:")
print("Z:", Z_point_estimate)
print(f"Var(Z):", var_Z)

Exercise 3 with variables from slides:
Z: 1.6517829572307585
Var(Z): 0.0030492941868425216


In [ ]:
def stratified_sampling(samples, n_strata):
    samples_per_stratum = samples // n_strata
    i = np.arange(n_strata)
    
    U = np.random.uniform(0, 1, (n_strata, samples_per_stratum))
    Y = (i[:, None] + U) / n_strata 
    W = np.exp(Y)
    
    return W, Y

n_strata = 10
W, Y = stratified_sampling(samples, n_strata)

In [113]:
print("Exercise 4:")
print(f"Stratified sampling mean: {np.mean(W)}")
print(f"Number of strata: {n_strata}")

Exercise 4:
Stratified sampling mean: 1.7167698726833693
Number of strata: 10


In [120]:
# Exercise 5 stratified sampling using control variates
def stratified_sampling_with_cv(samples, n_strata):
    samples_per_stratum = samples // n_strata
    i = np.arange(n_strata)
    stratum_means = (i + 0.5) / n_strata
    
    U = np.random.uniform(0, 1, (n_strata, samples_per_stratum))
    Y = (i[:, None] + U) / n_strata 
    W = np.exp(Y)

    Z = np.zeros_like(W)
    stratum_variances = np.zeros(n_strata)
    
    for k in range(n_strata):
        cov = np.cov(W[k], Y[k])[0, 1]
        var_y = np.var(Y[k])
        c = -cov / var_y
        
        Z[k] = W[k] + c * (Y[k] - stratum_means[k])
        stratum_variances[k] = np.var(Z[k]) / samples_per_stratum

    total_variance = np.sum(stratum_variances) / (n_strata**2)
    
    return Z, total_variance

In [ ]:
# Exercise 5
Z, var_Z = stratified_sampling_with_cv(samples, n_strata)

print('Exercise 5: ')
print("Z:", np.mean(Z))
print("Var(Z):", var_Z)

Exercise 5: 
W: 1.7179554803491837
Var(W): 3.1731706092141887e-07


In [ ]:
# Exercise 7
from scipy.stats import norm

mu = 0
sigma_2 = 1
samples = 100
a = 2

crude_monte_carlo = lambda mu, sigma, samples: np.random.normal(mu, sigma, samples)


def importance_sampling(a, sigma, samples):
    f = lambda x: norm.pdf(x)
    g = lambda x: norm.pdf(x, loc=a, scale=sigma)
    
    g_samples = np.random.normal(a, sigma, samples)
    
    weights = f(g_samples) / g(g_samples)
    h = (g_samples > a).astype(float)
    
    estimate = np.mean(h * weights)
    variance = np.var(h * weights) / samples
    
    return estimate, variance

In [140]:
# Exercise 7
Z = crude_monte_carlo(mu, sigma_2, samples)
print("Crude monte carlo:")
for a in [0, 1, 2, 3, 4]:
    print("===============================")
    print(f"Odds of Z > a where a={a}: {(Z>a).mean()}")
    print("Variance:", (Z>a).var() / samples)
print("===============================")

Crude monte carlo:
Odds of Z > a where a=0: 0.51
Variance: 0.0024989999999999995
Odds of Z > a where a=1: 0.15
Variance: 0.001275
Odds of Z > a where a=2: 0.04
Variance: 0.00038399999999999996
Odds of Z > a where a=3: 0.01
Variance: 9.9e-05
Odds of Z > a where a=4: 0.0
Variance: 0.0


In [144]:
for a in range(4):
    for sigma in range(1,4):
        estimate, variance = importance_sampling(a, sigma, samples)
        print("===============================")
        print(f"Odds of Z > a where a={a} and sigma_2 = {sigma}: {estimate}")
        print("Variance:", variance)

Odds of Z > a where a=0 and sigma_2 = 1: 0.51
Variance: 0.0024989999999999995
Odds of Z > a where a=0 and sigma_2 = 2: 0.47183432257284375
Variance: 0.004332750054263219
Odds of Z > a where a=0 and sigma_2 = 3: 0.45202891037230164
Variance: 0.007671797510444788
Odds of Z > a where a=1 and sigma_2 = 1: 0.1519475364927777
Variance: 0.0004078230649432278
Odds of Z > a where a=1 and sigma_2 = 2: 0.18297639297409685
Variance: 0.0011308404724020159
Odds of Z > a where a=1 and sigma_2 = 3: 0.1662841721478388
Variance: 0.00150146891596583
Odds of Z > a where a=2 and sigma_2 = 1: 0.022129269144666078
Variance: 1.1260365677980657e-05
Odds of Z > a where a=2 and sigma_2 = 2: 0.02609550854135541
Variance: 3.4641585783224095e-05
Odds of Z > a where a=2 and sigma_2 = 3: 0.03131464784283799
Variance: 6.201495844782132e-05
Odds of Z > a where a=3 and sigma_2 = 1: 0.001085850433735574
Variance: 3.4846004140637864e-08
Odds of Z > a where a=3 and sigma_2 = 2: 0.0012309185253518394
Variance: 8.95528224749

In [153]:
# Exercise 8
def importance_sampling_exp(samples, _lambda):
    Y = np.random.exponential(scale=1/_lambda, size=samples)
    W = np.exp(Y) / (_lambda * np.exp(-_lambda * Y))
    estimate = np.mean(W * (Y <= 1))
    variance = np.var(W * (Y <= 1)) / samples
    return estimate, variance

In [181]:
optimal_lambda = None
lowest_var = np.inf

for _lambda in np.arange(1,2,0.1):
    est, var = importance_sampling_exp(100, _lambda)
    if var < lowest_var:
        lowest_var = var
        optimal_lambda = _lambda

print(f"Optimal lambda ~ {optimal_lambda} gives estimate {est}, with variance {var}")

Optimal lambda ~ 1.3000000000000003 gives estimate 1.7769066702253646, with variance 0.03903824071782655


In [73]:
# Exercise 8
def importance_sampling_exp(n_samples=10000, lambd=1):
    """Importance sampling with exponential proposal"""
    # Sample from exponential
    Y = np.random.exponential(scale=1/lambd, size=n_samples)
    # Only keep samples in [0,1]
    Y = Y[Y <= 1]
    if len(Y) == 0:
        return 0, 0  # Edge case when lambda is too large
    
    # Calculate weights
    f = 1  # Uniform PDF
    g = lambd * np.exp(-lambd * Y)  # Exponential PDF
    h = np.exp(Y)
    W = h * f / g
    
    # Correct for truncation to [0,1]
    correction = 1 - np.exp(-lambd)  # P(Y ≤ 1)
    return np.mean(W) * correction, np.var(W)/len(Y) * correction**2

# Find optimal lambda by minimizing variance
lambdas = np.linspace(0.1, 10, 50)
variances = []
for l in lambdas:
    _, var = importance_sampling_exp(lambd=l)
    variances.append(var)
    
optimal_lambda = lambdas[np.argmin(variances)]
print(f"\nOptimal lambda: {optimal_lambda:.2f}")

# Compare with true integral (e^1 - e^0 ≈ 1.71828)
est, var = importance_sampling_exp(lambd=optimal_lambda)
print(f"IS estimate: {est:.6f}, True value: {np.exp(1)-1:.6f}")
print(f"Variance: {var:.2e}")


Optimal lambda: 0.50
IS estimate: 1.729972, True value: 1.718282
Variance: 1.46e-04


In [ ]:
# Exercise 9
